In [1]:
import os
import exifreader
import re
import pprint
from   datetime import datetime
import pandas   as     pd
import numpy    as     np
from   hashlib  import md5
import shutil

In [25]:
store_path ='/mnt/storevg/store/replicated/Pictures/'
remote_path='/media/ryan/Seagate Backup Plus Drive/Mac Bck'
remote_path='/media/ryan/Seagate Backup Plus Drive/'
remote_name='Seagate Backup Plus Drive'

In [14]:
file_extensions = {'JPEG':['jpeg','jpg'],
                   'CR2':['CR2'], 
                   'ARW':['arw'],
                   'DNG':['dng'],
                   'TIFF':['tiff']
                  }

re_list = []
for extension in file_extensions:
    for ext in file_extensions[extension]:
        re_list.append(re.compile("{}$".format(ext),re.I))
    

In [15]:
def get_exif_value(tag_string,tags,postProcessing=lambda x:x):
    if tag_string in tags:
        value = tags[tag_string]
        value = postProcessing(value)
    else:
        return 'No value'
    return value

In [16]:
exif_date_to_datetime = lambda x: datetime.strptime(x.printable,'%Y:%m:%d %H:%M:%S')

exif_to_get_list = [{'exif_string':'EXIF DateTimeOriginal','function':exif_date_to_datetime}]

def make_exif_stats(file,exif_to_get_list):
    tags = exifreader.process_file(file)
    tags_dict = {}
    for exif_dict in exif_to_get_list:
        value = get_exif_value(exif_dict['exif_string'],tags,exif_dict['function'])
        tags_dict[exif_dict['exif_string']] = value  
    return tags_dict

In [21]:
def make_df_dir(dir_path,file_extension_re,exif_to_get_list,verbose=0,max_files=-1,check_md5=False):
    file_dict_list = []
    for root, subFolder, files in os.walk(dir_path):
        if verbose>0: print("root: {} nFiles: {}".format(root, len(files)))
        for item in files:
            if verbose>2: print("\t item: {}".format(item))
            re_search = [re.search(item) for re in file_extension_re]
            matched_one =  not all([match is None for match in re_search])
            if matched_one:
                if verbose>1: print("\t matched item: {}".format(item))
                file_dict = {}
                file_dict["item"] = item
                #basic file stats
                fileNamePath = str(os.path.join(root,item))
                file_dict["fileNamePath"]  =  fileNamePath
                file_dict['modify_date']   =  datetime.fromtimestamp(os.path.getmtime(fileNamePath))
                file_dict['file_size']     =  os.path.getsize(fileNamePath)
                file_dict['access_time']   =  datetime.now()
                #exif stats
                get_exif = len(exif_to_get_list)>0
                if (get_exif or check_md5):
                    with open(fileNamePath, 'rb') as f:
                        if get_exif:
                            tags_dict = make_exif_stats(f,exif_to_get_list)
                            file_dict.update(tags_dict)
                        if check_md5:
                            f_bytes = f.read()
                            file_dict['md5_hexdigest'] = md5(f_bytes).hexdigest()
                            
                file_dict_list.append(file_dict)
                if max_files>0 and len(file_dict_list)>=max_files: return pd.DataFrame(file_dict_list)
                
    return pd.DataFrame(file_dict_list)


In [29]:
store_df = make_df_dir(store_path,re_list,exif_to_get_list, verbose=1,max_files=-1,check_md5=True)

root: /mnt/storevg/store/replicated/Pictures/ nFiles: 12943
root: /mnt/storevg/store/replicated/Pictures/2014 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2014/11 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2014/11/07 nFiles: 158
root: /mnt/storevg/store/replicated/Pictures/2014/03 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2014/03/09 nFiles: 4
root: /mnt/storevg/store/replicated/Pictures/2014/06 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2014/06/14 nFiles: 2
root: /mnt/storevg/store/replicated/Pictures/2014/05 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2014/05/31 nFiles: 2
root: /mnt/storevg/store/replicated/Pictures/2014/10 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2014/10/27 nFiles: 4
root: /mnt/storevg/store/replicated/Pictures/2014/10/11 nFiles: 11
root: /mnt/storevg/store/replicated/Pictures/2014/10/10 nFiles: 37
root: /mnt/storevg/store/replicated/Pictures/2014/10/21 nFiles: 56
root: /mnt/storevg/store/replica

Possibly corrupted field Tag 0xDB24 in MakerNote IFD


root: /mnt/storevg/store/replicated/Pictures/2017/11/13 nFiles: 4
root: /mnt/storevg/store/replicated/Pictures/2017/06 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2017/06/03 nFiles: 303
root: /mnt/storevg/store/replicated/Pictures/2017/05 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2017/05/07 nFiles: 6
root: /mnt/storevg/store/replicated/Pictures/2017/05/30 nFiles: 23
root: /mnt/storevg/store/replicated/Pictures/2017/05/02 nFiles: 7
root: /mnt/storevg/store/replicated/Pictures/2017/10 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2017/10/14 nFiles: 20
root: /mnt/storevg/store/replicated/Pictures/2017/08 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2017/08/21 nFiles: 170
root: /mnt/storevg/store/replicated/Pictures/2017/01 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2017/01/31 nFiles: 2
root: /mnt/storevg/store/replicated/Pictures/2017/01/03 nFiles: 31
root: /mnt/storevg/store/replicated/Pictures/2017/01/02 nFiles: 43
root: /mnt/storev

root: /mnt/storevg/store/replicated/Pictures/2015/12/04 nFiles: 83
root: /mnt/storevg/store/replicated/Pictures/2015/12/19 nFiles: 4
root: /mnt/storevg/store/replicated/Pictures/2015/12/24 nFiles: 19
root: /mnt/storevg/store/replicated/Pictures/2015/12/12 nFiles: 4
root: /mnt/storevg/store/replicated/Pictures/2018 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2018/08 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2018/08/10 nFiles: 409


Possibly corrupted field Tag 0xDD59 in MakerNote IFD


root: /mnt/storevg/store/replicated/Pictures/2018/02 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2018/02/02 nFiles: 54
root: /mnt/storevg/store/replicated/Pictures/NoDateTimeOriginal nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/NoDateTimeOriginal/2014 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/NoDateTimeOriginal/2014/10 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/NoDateTimeOriginal/2014/10/18 nFiles: 2
root: /mnt/storevg/store/replicated/Pictures/NoDateTimeOriginal/2014/04 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/NoDateTimeOriginal/2014/04/16 nFiles: 2
root: /mnt/storevg/store/replicated/Pictures/NoDateTimeOriginal/2014/12 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/NoDateTimeOriginal/2014/12/06 nFiles: 2
root: /mnt/storevg/store/replicated/Pictures/NoDateTimeOriginal/2017 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/NoDateTimeOriginal/2017/01 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/NoDateTime

root: /mnt/storevg/store/replicated/Pictures/tmp/2020/12/17 nFiles: 18
root: /mnt/storevg/store/replicated/Pictures/tmp/2020/12/12 nFiles: 169
root: /mnt/storevg/store/replicated/Pictures/tmp/2012 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/tmp/2012/1 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/tmp/2012/1/2 nFiles: 38
root: /mnt/storevg/store/replicated/Pictures/tmp/2012/1/1 nFiles: 251
root: /mnt/storevg/store/replicated/Pictures/2013 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2013/11 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2013/11/09 nFiles: 2
root: /mnt/storevg/store/replicated/Pictures/2013/11/28 nFiles: 2
root: /mnt/storevg/store/replicated/Pictures/2013/08 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2013/08/09 nFiles: 4
root: /mnt/storevg/store/replicated/Pictures/2013/12 nFiles: 0
root: /mnt/storevg/store/replicated/Pictures/2013/12/11 nFiles: 4
root: /mnt/storevg/store/replicated/Pictures/2013/12/09 nFiles: 2
root: /mn

In [42]:
remote_df = make_df_dir(remote_path,re_list,exif_to_get_list, verbose=1,max_files=-1,check_md5=True)

root: /media/ryan/Seagate Backup Plus Drive/ nFiles: 6
 nFiles: 0ia/ryan/Seagate Backup Plus Drive/.HFS+ Private Directory Data
root: /media/ryan/Seagate Backup Plus Drive/2020_oct_27_backup_sdcard nFiles: 1
root: /media/ryan/Seagate Backup Plus Drive/2020_oct_27_backup_sdcard/Untitled nFiles: 1
root: /media/ryan/Seagate Backup Plus Drive/2020_oct_27_backup_sdcard/Untitled/.fseventsd nFiles: 1
root: /media/ryan/Seagate Backup Plus Drive/2020_oct_27_backup_sdcard/Untitled/DCIM nFiles: 0
root: /media/ryan/Seagate Backup Plus Drive/2020_oct_27_backup_sdcard/Untitled/DCIM/100EOS5D nFiles: 445
root: /media/ryan/Seagate Backup Plus Drive/2020_oct_27_backup_sdcard/Untitled/DCIM/EOSMISC nFiles: 1
root: /media/ryan/Seagate Backup Plus Drive/2020_oct_27_backup_sdcard/Untitled/MISC nFiles: 0
root: /media/ryan/Seagate Backup Plus Drive/Galactic-Astronomy nFiles: 12
root: /media/ryan/Seagate Backup Plus Drive/Galactic-Astronomy/.git nFiles: 7
root: /media/ryan/Seagate Backup Plus Drive/Galactic-Ast

root: /media/ryan/Seagate Backup Plus Drive/Mac Bck nFiles: 1
root: /media/ryan/Seagate Backup Plus Drive/Mac Bck/Desktop nFiles: 1
root: /media/ryan/Seagate Backup Plus Drive/Mac Bck/Desktop/w_ideal nFiles: 18
root: /media/ryan/Seagate Backup Plus Drive/Mac Bck/Desktop/w_ideal/atrium1 nFiles: 5
root: /media/ryan/Seagate Backup Plus Drive/Mac Bck/Desktop/w_ideal/atrium2 nFiles: 8
root: /media/ryan/Seagate Backup Plus Drive/Mac Bck/Desktop/w_ideal/bathroom1 nFiles: 7
root: /media/ryan/Seagate Backup Plus Drive/Mac Bck/Desktop/w_ideal/dinningroom1 nFiles: 8
root: /media/ryan/Seagate Backup Plus Drive/Mac Bck/Desktop/w_ideal/dinningroom2 nFiles: 7
root: /media/ryan/Seagate Backup Plus Drive/Mac Bck/Desktop/w_ideal/downstairsbath nFiles: 5
root: /media/ryan/Seagate Backup Plus Drive/Mac Bck/Desktop/w_ideal/downstairsbed1 nFiles: 5
root: /media/ryan/Seagate Backup Plus Drive/Mac Bck/Desktop/w_ideal/downstairsbed2 nFiles: 2
root: /media/ryan/Seagate Backup Plus Drive/Mac Bck/Desktop/w_ideal/

PermissionError: [Errno 13] Permission denied: '/media/ryan/Seagate Backup Plus Drive/Mac Bck/Photos Library.photoslibrary/Photos Library.photoslibrary/Masters/2014/10/28/20141028-154804/_DSC6951.ARW'

In [ ]:
remote_df

In [ ]:
store_df.to_csv("on_store.csv")
remote_name_no_space = remote_name.replace(' ','_')
remote_df.to_csv("on_{}.csv".format(remote_name_no_space))

In [38]:
def get_only_on_remote_pd(store_df,remote_df,on=['item', 'file_size'],verbose=0):    
    merge_pd = remote_df.merge(store_df,
                    on=on,
                    how='outer',indicator=True)
    both_pd = merge_pd[(merge_pd._merge=='both')]
    only_remote_pd = merge_pd[(merge_pd._merge=='left_only')]
    nStore,nRemote,nRemoteOnly = store_df.shape[0],remote_df.shape[0],only_remote_pd.shape[0]
    nStoreOnly = merge_pd[(merge_pd._merge=='right_only')].shape[0]
    nBoth = both_pd.shape[0]
    nTotal = merge_pd.shape[0]
    if verbose: print("total: {} store : {}, remote: {}, only on remote: {}, only on store: {}, nBoth: {}".format(nTotal,nStore,nRemote,nRemoteOnly,nStoreOnly,nBoth))
    return only_remote_pd,both_pd


In [39]:
remote_df

""


In [34]:
must_match = ['item', 'file_size','md5_hexdigest']

In [35]:
only_remote_pd,both_pd = get_only_on_remote_pd(store_df,remote_df,verbose=True, on=must_match)

KeyError: 'item'

In [36]:
def copy_df_files(df, target_path, file_name_format):
    nToTransfer = only_remote_pd.shape[0]
    for index, row in only_remote_pd.iterrows():
        file_dict = row.to_dict()
        file_name_origin = row.fileNamePath_x
        item = row.item
        modify_date = row.modify_date_x
        with open(file_dict['fileNamePath_x'],'rb') as f:
            tags_dict = make_exif_stats(f,exif_to_get_list)
        file_dict.update(tags_dict)
        for file in file_dict:
            print(type(file_dict[file]))
        print(file_dict)
        break

In [37]:
copy_df_files(only_remote_pd, store_path, "")

<class 'str'>
<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'int'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'float'>
<class 'pandas._libs.tslibs.nattype.NaTType'>
<class 'pandas._libs.tslibs.nattype.NaTType'>
<class 'str'>
<class 'datetime.datetime'>
{'item': '8D5A1892.CR2', 'fileNamePath_x': '/media/ryan/Seagate Backup Plus Drive/2020_oct_27_backup_sdcard/Untitled/DCIM/100EOS5D/8D5A1892.CR2', 'modify_date_x': Timestamp('2020-08-27 15:01:23'), 'file_size': 30001669, 'access_time_x': Timestamp('2021-01-13 20:24:30.598808'), 'fileNamePath_y': nan, 'modify_date_y': NaT, 'access_time_y': NaT, '_merge': 'left_only', 'EXIF DateTimeOriginal': datetime.datetime(2020, 8, 27, 15, 1, 21)}
